In [1]:
import numpy as np
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, create_optimizer
import os


In [2]:
# ---------------------- Preparing Data SET ----------------------
# Step 1: CreatING dataset of simple math riddles
riddles = [
    {"text": "Riddle: What number do you get if you multiply all the numbers on a phone keypad together? Solution: 0"},
    {"text": "Riddle: I am a number. Double me and add 6, and you get 14. What number am I? Solution: 4"},
    {"text": "Riddle: I am a two-digit number. My tens digit is twice my ones digit. What number am I? Solution: 42"},
    {"text": "Riddle: The sum of two consecutive numbers is 15. What are the numbers? Solution: 7 and 8"},
    {"text": "Riddle: If you subtract 4 from me, then divide by 2, you get 3. What number am I? Solution: 10"},
    {"text": "Riddle: I am a three-digit number. My ones digit is 3 times my hundreds digit. My tens digit is 5 more than my hundreds digit. What number am I? Solution: 193"},
    {"text": "Riddle: What is the sum of the first 10 natural numbers? Solution: 55"},
    {"text": "Riddle: If you multiply me by myself, you get 49. What number am I? Solution: 7"},
    {"text": "Riddle: The sum of my digits is 9, and I am a two-digit number. If you reverse my digits, I decrease by 27. What number am I? Solution: 54"},
    {"text": "Riddle: The sum of two numbers is 10, and their product is 24. What are the numbers? Solution: 4 and 6"},
    {"text": "Riddle: I am a two-digit number. If you square me and subtract 11, you get 100. What number am I? Solution: 11"},
    {"text": "Riddle: I am an even number. If you divide me by 2, you get 12. What number am I? Solution: 24"},
    {"text": "Riddle: The sum of my digits is 7. If you reverse my digits, I become a multiple of 7. What number am I? Solution: 34"},
    {"text": "Riddle: If you add 9 to me and then divide by 3, you get my original number. What number am I? Solution: 9"},
    {"text": "Riddle: A farmer has 17 sheep, but all but 9 run away. How many sheep are left? Solution: 9"},
    {"text": "Riddle: I am a two-digit number. The difference between my digits is 4, and my digits multiply to give 45. What number am I? Solution: 59"},
    {"text": "Riddle: The sum of my digits is 12, and I am a three-digit number that is a multiple of 3. What number am I? Solution: 156"},
    {"text": "Riddle: I am a four-digit number. My thousands digit is twice my hundreds digit. My ones digit is half my tens digit. What number am I? Solution: 4286"},
    {"text": "Riddle: I am a prime number between 40 and 50. What number am I? Solution: 47"},
    {"text": "Riddle: I am a fraction. If you add 2 to my numerator, I become 3/4. If you add 3 to my denominator, I become 1/2. What fraction am I? Solution: 1/2"},
    {"text": "Riddle: A clock shows 3:15. What is the angle between the hour and minute hands? Solution: 7.5 degrees"},
    {"text": "Riddle: I am a three-digit number. The sum of my digits is 8, and my ones digit is twice my tens digit. What number am I? Solution: 215"},
    {"text": "Riddle: I am a two-digit number. If you multiply my digits, you get 18, and their sum is 9. What number am I? Solution: 36"},
    {"text": "Riddle: The difference between a two-digit number and the number formed by reversing its digits is 36. If the sum of the digits is 12, what is the number? Solution: 84"},
    {"text": "Riddle: I am a number between 30 and 40. The sum of my digits is 8. If you subtract 9 from me, the result is divisible by 7. What number am I? Solution: 34"},
    {"text": "Riddle: The sum of two consecutive odd numbers is 40. What are the numbers? Solution: 19 and 21"},
    {"text": "Riddle: I am a number. If you multiply me by 5 and subtract 4, you get twice my square. What number am I? Solution: 4"},
    {"text": "Riddle: A train is traveling at 60 mph. How far does it travel in 30 minutes? Solution: 30 miles"},
    {"text": "Riddle: If you add 8 to me and then divide by 4, you get 5. What number am I? Solution: 12"},
    {"text": "Riddle: If you subtract me from 20, the result is double my value. What number am I? Solution: 10"}
]

# Save the riddles to a text file
with open('math_riddles.txt', 'w') as f:
    for riddle in riddles:
        f.write(riddle['text'] + '\n')


In [7]:
# Load model and tokenizer
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = TFGPT2LMHeadModel.from_pretrained(model_name)

# Ensure tokenizer has required tokens
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Prepare dataset
def load_dataset(file_path, tokenizer, block_size=128):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    tokenized_text = tokenizer.encode(text)
    examples = [tokenized_text[i:i + block_size] for i in range(0, len(tokenized_text) - block_size, block_size)]
    return tf.data.Dataset.from_tensor_slices(examples)

dataset = load_dataset('math_riddles.txt', tokenizer)

def add_labels(examples):
    attention_mask = [1] * len(examples)
    return {"input_ids": examples, "attention_mask": attention_mask, "labels": examples}

dataset = dataset.map(add_labels)

# Batch the dataset
batch_size = 2
dataset = dataset.batch(batch_size)


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [10]:
# ---------------------- Training Model ----------------------
# Set up training parameters
training_args = {
    'learning_rate': 3e-5,
    'training_epochs': 3,
    'weight_decay': 0.01,
}

total_steps = len(dataset) * training_args['training_epochs']
warmup_steps = int(0.1 * total_steps)

optimizer, lr_schedule = create_optimizer(
    init_lr=training_args['learning_rate'],
    num_train_steps=total_steps,
    num_warmup_steps=warmup_steps,
    weight_decay_rate=training_args['weight_decay'],
)

model.compile(optimizer=optimizer)

# Train the model
model.fit(dataset, epochs=training_args['training_epochs'])

# Save the model
model.save("math_riddle.keras")

Epoch 1/3
4/4 [==============================] - 35s 265ms/step - loss: 1.0881
Epoch 2/3
4/4 [==============================] - 1s 265ms/step - loss: 0.8965
Epoch 3/3
4/4 [==============================] - 1s 278ms/step - loss: 0.7882


In [11]:
# ---------------------- Generating Math Riddles ----------------------
def generate_riddle(model, tokenizer, prompt="Riddle: ", max_length=100, temperature=0.7):
    input_ids = tokenizer.encode(prompt, return_tensors='tf')
    attention_mask = tf.ones_like(input_ids)
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=temperature,
        top_p=0.8,
        top_k=50,
        do_sample=True,
        no_repeat_ngram_size=2,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [13]:
# Generate 5 test riddles
print("Generating test riddles...")
for i in range(5):
    riddle = generate_riddle(model, tokenizer)
    print(f"Test Riddle {i+1}: {riddle}")
    print("-" * 50)

Generating test riddles...
Test Riddle 1: Riddle: _____. What number am I? Solution: 14
Riddles: I am a number. My product is a multiple of 4. How many times am my product? Riddle 2: If you multiply me by 4, you get 9. If I were a prime number, how many more times would you add?Solution: 27
Solution (Riddler): If the sum of two consecutive odd numbers is odd, then divide by 3, and you obtain 10. Riddler
--------------------------------------------------
Test Riddle 2: Riddle: _____. The sum of my digits is 4. What number am I? Solution: 6
Riddles: I am a number. If you subtract 3 from me, you get 4, and if you add 9 to me I become 5. How many digits am my number?Solution: 7
Solution : I'm a multiple of 7.
The Riddle : The difference between two numbers is 10. My digits are 4 and my ones are 3. Which number are my
--------------------------------------------------
Test Riddle 3: Riddle: _____ is a number. What is its sum? Solution: 3
Riddles: I am a two-digit number, and my product is 4